# IMPORTING

In [50]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols

In [51]:
%store -r data_2

# Alterations: Engineered

In [53]:
data_3 = data_2

In [54]:
data_3['binary_reno'] = data_2.yr_renovated

In [55]:
#Whether or not the house has been renovated, regardless of year
data_3.binary_reno = data_2.binary_reno.apply(lambda x: 1 if x != 0 else 0)

In [56]:
data_3.binary_reno.unique()

array([0, 1])

In [57]:
#Combining renovation and number of floors as sqft_living multiplier
data_3['floor_reno_mult'] = ((data_3.binary_reno + 1) * (data_3.floors)) * data_3.sqft_living

In [58]:
data_3 = data_3.drop(columns = 'binary_reno')

# Scaling

In [59]:
# scale_data_3 = data_3

In [60]:
# ss = StandardScaler()

In [61]:
# x_columns = list(scale_data_3.columns)
# X=scale_data_3[x_columns]

In [62]:
# X_scaled = ss.fit_transform(X)

In [63]:
# scale_data_3.columns

In [64]:
# scale_data_3['scaled_price'] = X_scaled[:, 0]
# scale_data_3['scaled_bedrooms'] = X_scaled[:, 1]
# scale_data_3['scaled_bathrooms'] = X_scaled[:, 2]
# scale_data_3['scaled_sqft_living'] = X_scaled[:, 3]
# scale_data_3['scaled_sqft_lot'] = X_scaled[:, 4]
# scale_data_3['scaled_floors'] = X_scaled[:, 5]
# scale_data_3['scaled_waterfront'] = X_scaled[:, 6]
# scale_data_3['scaled_view'] = X_scaled[:, 7]
# scale_data_3['scaled_condition'] = X_scaled[:, 8]
# scale_data_3['scaled_grade'] = X_scaled[:, 9]
# scale_data_3['scaled_sqft_above'] = X_scaled[:, 10]
# scale_data_3['scaled_sqft_living15'] = X_scaled[:, 11]
# scale_data_3['scaled_sqft_lot15'] = X_scaled[:, 12]
# scale_data_3['scaled_date_year'] = X_scaled[:, 13]
# scale_data_3['scaled_date_month'] = X_scaled[:, 14]
# scale_data_3['scaled_date_day'] = X_scaled[:, 15]
# scale_data_3['scaled_rel_to_neighbors'] = X_scaled[:, 16]
# scale_data_3['scaled_size_quality'] = X_scaled[:, 17]
# scale_data_3['scaled_property_score'] = X_scaled[:, 18]
# scale_data_3['scaled_floor_reno_mult'] = X_scaled[:, 19]

In [65]:
# data_3.zipcode

In [66]:
# data_3.columns #WHYYYYY

In [67]:
lr_scaled = ols(formula='price~scaled_bedrooms+scaled_bathrooms+scaled_sqft_living+scaled_sqft_lot+scaled_floors+scaled_waterfront+scaled_view+scaled_condition+scaled_grade+scaled_sqft_above+scaled_sqft_living15+scaled_sqft_lot15+scaled_date_year+scaled_date_month+scaled_date_day+scaled_rel_to_neighbors+scaled_size_quality+scaled_property_score+scaled_floor_reno_mult', data=scale_data_3).fit()

In [68]:
lr_scaled.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     2327.
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        11:20:04   Log-Likelihood:            -2.3607e+05
No. Observations:               17290   AIC:                         4.722e+05
Df Residuals:                   17272   BIC:                         4.723e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                5.407e+05   1565.887    345.325      0.000    5.38e+05    5.44e+05
scaled_bedrooms         -3.556e+04   2032.398    -17.498      0.000   -3.95e+04   -3.16e+04
scaled_bathrooms          3.36e+04   2865.967     11.723      0.000     2.8e+04    3.92e+04
scaled_sqft_living       8.511e+04   1824.144     46.657      0.000    8.15e+04    8.87e+04
scaled_sqft_lot          4109.6648   2217.461      1.853      0.064    -236.783    8456.113
scaled_floors             -21.2080   2204.141     -0.010      0.992   -4341.548    4299.132
scaled_waterfront        5.313e+04   1724.079     30.815      0.000    4.97e+04    5.65e+04
scaled_view              3.751e+04   1871.401     20.044      0.000    3.38e+04    4.12e+04
scaled_condition         2.073e+04   1736.721     11.938      0.000    1.73e+04    2.41e+04
scaled_grade             1.131e+05   2899.112     39.012      0.000    1.07e+05    1.19e+05
scaled_sqft_above        7.973e+04   1954.110     40.800      0.000    7.59e+04    8.36e+04
scaled_sqft_living15      2.75e+04   1781.625     15.435      0.000     2.4e+04     3.1e+04
scaled_sqft_lot15       -7.355e+04   2411.411    -30.502      0.000   -7.83e+04   -6.88e+04
scaled_date_year         7758.2868   1677.086      4.626      0.000    4471.029     1.1e+04
scaled_date_month       -3.646e-12   8.02e-13     -4.548      0.000   -5.22e-12   -2.07e-12
scaled_date_day          7.776e+04   1657.248     46.921      0.000    7.45e+04     8.1e+04
scaled_rel_to_neighbors -1.766e+04   1906.571     -9.260      0.000   -2.14e+04   -1.39e+04
scaled_size_quality      1.892e+04   2688.444      7.038      0.000    1.37e+04    2.42e+04
scaled_property_score   -9081.3178   2237.681     -4.058      0.000   -1.35e+04   -4695.237
scaled_floor_reno_mult   1.358e+04   1570.536      8.648      0.000    1.05e+04    1.67e+04
==============================================================================
Omnibus:                    15386.094   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1745892.868
Skew:                           3.809   Prob(JB):                         0.00
Kurtosis:                      51.636   Cond. No.                     7.49e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.58e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Scaling Adjustments

In [69]:
data_3.grade.value_counts() / len(data_3)

7     0.415442
8     0.280393
9     0.121400
6     0.095315
10    0.051648
11    0.018450
5     0.011105
12    0.004164
4     0.001272
13    0.000636
3     0.000116
1     0.000058
Name: grade, dtype: float64

In [70]:
# Remove single row with grade=1 because can't use for Stratified Shuffle Split
data_3 = data_3[data_3.grade != 1]

# New Dataframe

In [71]:
%store data_3

Stored 'data_3' (DataFrame)


# Stratified Split

In [74]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data_3, data_3['grade']):
    strat_train_set = data_3.loc[data_3.index.intersection(train_index)]
    strat_test_set = data_3.loc[data_3.index.intersection(test_index)]


# PREPARE DATA FOR MACHINE LEARNING

In [75]:
x_train = strat_train_set.drop('price', axis=1)
y_train = strat_train_set['price'].copy()

# LINEAR REGRESSION

In [76]:
lr = LinearRegression()

lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [77]:
def array_to_dict(labels,coeffs):
    '''
    This function takes two arrays as arguments and turns those 
    arrays into a dictionary so that it is easier to interpret the 
    coefficients from the sklearn analysis
    '''
    one_list = list(labels)
    two_list = list(coeffs)
    res = {} 
    for key in two_list: 
        for name in one_list: 
            res[key] = name 
            one_list.remove(name) 
            break 
    return res

In [78]:
coefficients = lr.coef_
variables = np.array(x_train.columns)

array_to_dict(coefficients, variables)

{'bedrooms': -9882.723669830753,
 'bathrooms': 45288.634364065074,
 'sqft_living': -325.79470471162193,
 'sqft_lot': 0.2806818948230508,
 'floors': -17671.018924008764,
 'waterfront': 576084.0425726292,
 'view': 47221.48212540723,
 'condition': -57873.48878445818,
 'grade': -16301.586072234628,
 'sqft_above': -167.84337676541858,
 'sqft_basement': -157.95132799121492,
 'yr_built': -1831.5497184162082,
 'yr_renovated': 2.4771607172448555,
 'zipcode': -561.5224348524519,
 'lat': 609019.326522398,
 'long': -193991.38917413438,
 'sqft_living15': 33.58060258449922,
 'sqft_lot15': -0.41524205930307634,
 'date_year': 38769.04613473896,
 'date_month': 1868.657464744708,
 'date_day': -238.2441569715204,
 'rel_to_neighbors': -10972.005539419924,
 'size_quality': 49.63926828449761,
 'property_score': -0.33846484258789716,
 'floor_reno_mult': 22.372179043105785}

In [79]:
lr.intercept_

-71570567.28993994

In [80]:
y_pred = lr.predict(x_train)

In [81]:
lr.score(x_train, y_train)

0.7500240032971464

In [82]:
lr_rsme = mean_squared_error(y_train, y_pred)
lr_rmse = np.sqrt(lr_rsme)

rmse_dollars = '${:,.2f}'.format(lr_rmse)
rmse_dollars

'$185,777.71'